### PPMI Parkinsons set

In the extracted files count folder: 
~9800 count files from Phase1 and PHase 2 of IR2 and IR3 (checked for IR1 ... )
I split them into different folders to make scripts run easier:

phase1:
    IR2:
    IR3:
phase2:
    IR2
    IR3

I want to use IR3 release as its the latest release of the data BUT kind of get an understanding of the dataset by extracting both ... 


Sample names are period delimited with the following fields:
    FIELD 1: study and release version (PPMI-phase1-IR1) -> removed 
    FIELD 2: individualID or PATNO (e.g. 3385)
    FIELD 3: visit (e.g. V08)
    FIELD 4: sampleID (e.g. PP00175168)
    FIELD 5: sequencing facility ID (e.g. 5104-SL-0001)
    FIELD 6: assay (longRNA-NEBKAP)
    FIELD 7: analysis (salmon-gtf or starTG) 
    FIELD 8+: optional compression tar/gz and additional file sub-dividers


Visit Code
Visit Description
Number of Subjects
SC
Screening
5525
BL
Baseline
2986
V01
Month 3
8
V02
Month 6
46
V03
Month 9
0
V04
Month 12
1418
V05
Month 18
22
V06
Month 24
1150
V07
Month 30
0
V08
Month 36
87
V09
Month 42
0
V10
Month 48



In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from scipy.io import mmread
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))
from helper import assemble_single_featureCounts_outputs_to_df
import geo_utils
from geo_utils import read_prefixes_from_metadata_txt
from filepath import *
import glob
%autoreload 2

In [ ]:
file_list = glob.glob(os.path.join(PPMI_P1_IR3, "*.txt"))
len(file_list)

In [ ]:
# NOTE The follwing 2 operations took 50min each, i saved both in work ppmi and will continue from there on.
#p2_ir3_df = assemble_single_featureCounts_outputs_to_df(PPMI_P1_IR3) # depending on the input list this may take a long while
#P1_IR3_df.to_csv(BASE_PATH +  '/ppmi/combi_p1.csv')
#p2_ir3_df = assemble_single_featureCounts_outputs_to_df(PPMI_P2_IR3) # depending on the input list this may take a long while
#p2_ir3_df.to_csv(BASE_PATH +  '/ppmi/combi_p1.csv')

In [ ]:
#NOTE still takes like 5 min
p1_ir3_df = pd.read_csv(BASE_PATH +  '/ppmi/combi_p1.csv')
print(p1_ir3_df.shape)
p1_ir3_df.head()

In [ ]:
p2_ir3_df = pd.read_csv(BASE_PATH +  '/ppmi/combi_p2.csv')
print(p2_ir3_df.shape)
p2_ir3_df.head()

In [ ]:
ir3_combi_df =pd.concat([p1_ir3_df, p2_ir3_df], axis=1)
print(ir3_combi_df.shape)
ir3_combi_df = ir3_combi_df.iloc[:, 1:]
ir3_combi_df.head()

In [ ]:
#ir3_combi_df.to_csv(BASE_PATH + '/ppmi/combi.csv')
ir3_combi_df = pd.read_csv(BASE_PATH + '/ppmi/combi.csv')

In [ ]:
ir3_combi_df[ir3_combi_df['PATNO']==1009]

In [ ]:
#NOTE Investigation only 
'''from collections import Counter

split_entries=[]
columns_combi = ir3_combi_df.columns
for entry in columns_combi:
    split_entries.extend(entry.split('.'))
counter = Counter(split_entries)
# Sort occurrences in descending order
sorted_occurrences = counter.most_common()

# Print all occurrences in descending order
for element, count in sorted_occurrences:
    print(f"{element}: {count}")
'''
### OUT;
''' 
BL: 1530  # BASELINE 
V04: 873 Visit 4
V02: 861 Visit 2
V06: 843
V08: 543
POOL: 106
'''

In [ ]:
meta_ir3 = pd.read_csv(PPMI_IR3_META)
print(meta_ir3.shape)
meta_ir3.head()

#1009.POOL.0003202425.5104-SL-0095	

#PATNO.CLINICAL_EVENT.Specimen Bar Code.1.HudAlphaID
meta_ir3.head()

In [ ]:
meta_ir3['PATNO'].nunique()
meta_ir3_BL = meta_ir3[meta_ir3['CLINICAL_EVENT'] == 'BL']
meta_ir3_BL.shape

In [ ]:
# I try to sort out the Baseline (BL) columns as i want to start with non longitudinal data first
#TODO find out what POOL is ... according to the paper there are single cell poolings. Neither the data doc seemed to helphere: https://ida.loni.usc.edu/download/files/study/a30291c2-12ca-41fa-a1a4-da9fad1682b3/file/ppmi/PPMI_Data_User_Guide_v.1.2_20240627.pdf 


filtered_columns = [col for col in ir3_combi_df.columns if len(col.split('.')) > 1 and col.split('.')[1] == 'BL']
ir3_combi_df_BL = ir3_combi_df[filtered_columns]
ir3_combi_df_BL.columns = [col.split('.')[0] for col in ir3_combi_df_BL.columns]
ir3_combi_df_BL_T = ir3_combi_df_BL.T
ir3_combi_df_BL_T.columns = list(ir3_combi_df['Geneid'])
ir3_combi_df_BL_T['PATNO'] = ir3_combi_df_BL_T.index
print(ir3_combi_df_BL_T.shape)
ir3_combi_df_BL_T.head()



In [ ]:
#meta_ir3_BL['PATNO'] = meta_ir3_BL['PATNO'].astype(str)
meta_ir3['PATNO'] = meta_ir3['PATNO'].astype(str)
ir3_combi_df_BL_w_meta = pd.merge(meta_ir3, ir3_combi_df_BL_T, left_on='PATNO', right_on='PATNO', how='right') # As I wantz the baseline only
print(ir3_combi_df_BL_w_meta.shape)

ir3_combi_df_BL_w_meta.head()


In [ ]:
ir3_combi_df_BL_w_meta.to_csv(BASE_PATH + '/ppmi/ir3_bl_w_meta_combi.csv')

In [ ]:
#TODO Investigate if there are IR2 data apart from this here ... 